In [2]:
import pandas as pd
dfclass = pd.read_csv(r'UrbanSound8K.csv')
dfclass.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [3]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold
import librosa
import random
import gc


2023-12-02 19:42:12.242975: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 19:42:12.279223: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 19:42:12.280795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 19:42:13.036035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def load_raw_audio(file_path, sample_rate=22050):
    # Carrega o arquivo de áudio com librosa
    audio, _ = librosa.load(file_path, sr=sample_rate)
    return audio


In [5]:
FOLDS_PATH = "../UrbanSound8K/audio/"
all_audio_paths = []
all_labels = []

# Supondo que você tenha um DataFrame df com as colunas 'file_path' e 'label'
# Organize os dados de cada fold em listas separadas

i=0
# Organize os dados de cada fold em listas separadas
fold_data = {f'fold{i}': {'all_paths': [], 'paths': [], 'labels': []} for i in range(1, 11)}

for index, row in dfclass.iterrows():
    i+=1
    fold = row['fold']
    file_path = os.path.join(FOLDS_PATH, f"fold{fold}", row["slice_file_name"])
    fold_data[f"fold{fold}"]['all_paths'].append(file_path)
    fold_data[f"fold{fold}"]['labels'].append(row['classID'])

print(i)
i=0
print(len(fold_data['fold1']['all_paths']))

# Dividir aleatoriamente cada fold em metades
for fold, data in fold_data.items():
    combined = list(zip(data['all_paths'], data['labels']))
    random.shuffle(combined)
    paths, labels = zip(*combined)
    split_index = len(paths)//10
    data['paths'] = paths[:split_index]
    data['labels'] = labels[:split_index]
    
    i+=1
print(i)
print(len(fold_data['fold1']['paths']))

8732
873
10
87


In [6]:
gc.collect()

0

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

def create_model(input_shape, num_classes=10):
    model = Sequential()
    model.add(Conv1D(32, 5, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(3))
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(3))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))  # Ajuste para o número de classes
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


2023-12-02 19:47:43.163360: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 19:47:43.217277: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 19:47:43.218770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 19:47:44.406098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
TARGET_LENGTH = 4*22050

def process_audio_paths(paths, target_length=TARGET_LENGTH):
    processed_audios = []
    for path in paths:
        audio = load_raw_audio(path)
        # Truncar ou preencher com zeros para obter um comprimento uniforme
        if len(audio) > target_length:
            audio = audio[:target_length]
        else:
            audio = np.pad(audio, (0, max(0, target_length - len(audio))), "constant")
        processed_audios.append(audio)
    return np.array(processed_audios)

results = []
histories=[]

for i in range(1, 11):
    print(f"Processando Fold {i}")
    
    # Processa os caminhos dos áudios
    test_paths = fold_data[f'fold{i}']['paths']
    test_labels = fold_data[f'fold{i}']['labels']
    train_paths = [path for fold, data in fold_data.items() if fold != f'fold{i}' for path in data['paths']]
    train_labels = [label for fold, data in fold_data.items() if fold != f'fold{i}' for label in data['labels']]
    
    X_train = process_audio_paths(train_paths)
    X_test = process_audio_paths(test_paths)
    y_train = np.array(train_labels)
    y_test = np.array(test_labels)
    
    # Dividir o conjunto de teste em metades iguais para teste e validação
    split_index = len(X_test) // 2
    X_val, y_val = X_test[:split_index], y_test[:split_index]
    X_test, y_test = X_test[split_index:], y_test[split_index:]
    
    # Criação e treinamento do modelo
    print(f"modelo {i}")
    model = create_model((TARGET_LENGTH, 1))  # Ajuste as dimensões conforme necessário
    from tensorflow.keras.callbacks import EarlyStopping

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
    print(f"acabei modelo{i}")
    # Avaliação do modelo
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    histories.append(history.history)
    results.append(accuracy_score(y_test, y_pred_classes))

    # Imprimir resultados do fold atual
    print(f"Fold {i} Accuracy: {results[-1]}")


Processando Fold 1
modelo 1


2023-12-02 19:42:46.190180: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-02 19:42:46.312991: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-12-02 19:42:46.454692: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 321060864 exceeds 10% of free system memory.
2023-12-02 19:42:46.519692: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 321060864 exceeds 10% of free system memory.
2023-12-02 19:42:46.576843: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocat

Epoch 1/10


2023-12-02 19:42:46.888797: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 275536800 exceeds 10% of free system memory.
2023-12-02 19:42:47.211393: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 321060864 exceeds 10% of free system memory.


In [ ]:
# Média das acurácias
average_accuracy = np.mean(results)
print(f"Average Accuracy: {average_accuracy}")

# Desvio padrão das acurácias
std_dev_accuracy = np.std(results)
print(f"Standard Deviation of Accuracy: {std_dev_accuracy}")


In [ ]:
import matplotlib.pyplot as plt

# Plotar gráficos para o último fold como exemplo
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
